In [35]:
import os
import shutil
import time
import re
import random
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# --- CONSTANTS ---
CARS_FOLDER = "cars"
PERSONNUMMER = ""
IF_URL = "https://www.if.se/privat/partner/nordea/forsakringar/fordon/bil"
CHROME_PATH = "/Applications/Google Chrome for Testing.app/Contents/MacOS/Google Chrome for Testing"
CHROME_VERSION = 125  # Match your installed Chrome version

# --- Setup UC Driver ---
options = uc.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--start-maximized")
options.binary_location = CHROME_PATH

driver = uc.Chrome(options=options, version_main=CHROME_VERSION)
wait = WebDriverWait(driver, 30)  # increased to 30s just in case

def get_insurance_cost(regnum):
    """Fetch insurance cost for a given 'regnum' from if.se (Nordea partner page).
       Returns digits only or 'N/A'."""
    try:
        print(f"[DEBUG] Going to IF URL: {IF_URL}")
        driver.get(IF_URL)
        # Let the page load thoroughly
        time.sleep(random.uniform(1, 2))

        # 1. Accept cookies if present
        try:
            cookie_btn = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "#onetrust-accept-btn-handler"))
            )
            time.sleep(0.5)
            driver.execute_script("arguments[0].click();", cookie_btn)
            print("[DEBUG] Cookie banner accepted.")
            time.sleep(1)
        except Exception as e:
            print(f"[DEBUG] No cookie banner or error accepting cookie: {e}")

        # 2. Fill in the registration number
        reg_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "input[placeholder='ABC123']")
            )
        )
        time.sleep(0.5)
        reg_input.clear()
        time.sleep(0.5)
        reg_input.send_keys(regnum)
        print(f"[DEBUG] Entered registration number: {regnum}")
        time.sleep(1)

        # 2a. Check if there's an error about invalid regnum
        error_selector = (
            By.CSS_SELECTOR,
            "#hero > div > div.op.if.content > os-entry > os-entry-template > "
            "div > os-entry-regno-input > div > os-entry-input > "
            "os-unified-text-input > div > div > div.error-text.force-show"
        )
        try:
            WebDriverWait(driver, 2).until(EC.visibility_of_element_located(error_selector))
            print(f"[DEBUG] Invalid regnum alert for: {regnum}")
            return "N/A"
        except:
            pass

        # 3. Fill in personal number
        pnr_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "input[placeholder='ååmmddnnnn']")
            )
        )
        time.sleep(0.5)
        pnr_input.clear()
        time.sleep(0.5)
        pnr_input.send_keys(PERSONNUMMER)
        print(f"[DEBUG] Entered personal number: {PERSONNUMMER}")
        time.sleep(1)

        # 4. Click 'Se ditt pris'
        continue_btn = wait.until(EC.element_to_be_clickable((
            By.CSS_SELECTOR,
            "#hero > div > div.op.if.content > os-entry > os-entry-template > div > os-spinner-button > button"
        )))
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", continue_btn)
        time.sleep(0.5)
        try:
            continue_btn.click()
            print("[DEBUG] Clicked 'Se ditt pris'.")
        except:
            driver.execute_script("arguments[0].click();", continue_btn)
            print("[DEBUG] Fallback click for 'Se ditt pris'.")
        time.sleep(3)

        # 5. Look for the price
        try:
            price_el = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.CSS_SELECTOR,
                    "body > div.if.app > main > os-main > os-main-template > "
                    "pd-price-bar > div > p > pd-price > span"
                ))
            )
            monthly_cost_text = price_el.get_attribute("innerText") or ""
            print(f"[DEBUG] Found price text: '{monthly_cost_text}'")
        except Exception as e:
            print(f"[DEBUG] Couldn't find price element: {e}")
            return "N/A"

        # 6. Convert text like "1 154 kr/mån" into "1154"
        monthly_cost_str = "".join(re.findall(r"\d+", monthly_cost_text))
        if monthly_cost_str:
            return monthly_cost_str
        else:
            return "N/A"

    except Exception as e:
        print(f"[DEBUG] Exception in get_insurance_cost: {e}")
        return "N/A"

def process_all_folders():
    """Iterates over subfolders in CARS_FOLDER, reads 'regnum.txt',
    tries to fetch the monthly cost, writes it to 'forsakring.txt'
    or removes folder on failure."""
    for folder in os.listdir(CARS_FOLDER):
        folder_path = os.path.join(CARS_FOLDER, folder)
        if not os.path.isdir(folder_path):
            continue

        reg_file = os.path.join(folder_path, "regnum.txt")
        if not os.path.exists(reg_file):
            print(f"[SKIP] No regnum.txt in '{folder}'")
            continue

        insurance_file = os.path.join(folder_path, "forsakring.txt")
        if os.path.exists(insurance_file):
            print(f"[SKIP] forsakring.txt already exists in '{folder}'")
            continue

        with open(reg_file, "r", encoding="utf-8") as f:
            regnum = f.read().strip().upper()

        if not regnum:
            print(f"[SKIP] Empty regnum.txt in '{folder}'")
            continue

        print(f"\n[INFO] Getting insurance for '{regnum}' in folder '{folder}'...")
        price = get_insurance_cost(regnum)

        if price != "N/A":
            time.sleep(0.5)  # Delay before writing
            with open(insurance_file, "w", encoding="utf-8") as f:
                f.write(price)
            print(f"[SAVED] {regnum} → {price} kr/månad in '{folder}'")
        else:
            print(f"[FAILED] Could not get price for {regnum}. Removing directory '{folder}'.")
            shutil.rmtree(folder_path, ignore_errors=True)
            continue

        # A shorter random delay before the next iteration
        time.sleep(random.uniform(3, 5))

# --- MAIN EXECUTION ---
try:
    process_all_folders()
finally:
    driver.quit()


[SKIP] forsakring.txt already exists in 'Volkswagen'
[SKIP] forsakring.txt already exists in 'Ford Kuga 2.0 TDCi AWD Powershift Trend Euro 6'
[SKIP] forsakring.txt already exists in 'Audi A3 Sportback 1.0 TFSI Comfort Euro 6'
[SKIP] forsakring.txt already exists in 'Citroën Berlingo Multispace 1.6 BlueHDi 100 8v ETG6 Euro 6'
[SKIP] forsakring.txt already exists in 'Skoda Superb Kombi 2.0 TDI 4x4 Elegance Euro 5'
[SKIP] forsakring.txt already exists in 'Volvo V90 Recharge T8 AWD Geartronic R-Design Euro6 Polestar'
[SKIP] forsakring.txt already exists in 'BMW X1 xDrive20d Sport line Euro 5'
[SKIP] forsakring.txt already exists in 'Tesla Model 3 Long Range AWD_Autopilot, Dator för FSD'
[SKIP] forsakring.txt already exists in 'BMW 116 i 5-dörrars M Sport Euro 6'
[SKIP] forsakring.txt already exists in 'BMW 318 d Sedan Advantage Euro 6'
[SKIP] forsakring.txt already exists in 'Opel Astra 1.4 Turbo Enjoy Euro 5'
[SKIP] forsakring.txt already exists in 'Audi Q7 3.0 TDI V6 quattro TipTronic S 